In [26]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.2


In [39]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import codecs
import fnmatch
import os
import sys
import tarfile
import tempfile
import unicodedata

import pandas
from six.moves import urllib
!pip install pydub
!pip install  ffmpeg
from os.path import splitext
from pydub import AudioSegment






In [0]:
LIBRI_SPEECH_URLS = {
    "train-clean-100":
        "http://www.openslr.org/resources/12/train-clean-100.tar.gz"
}

def download_and_extract(directory, url):
  """Download and extract the given split of dataset.
  Args:
    directory: the directory where to extract the tarball.
    url: the url to download the data file.
  """

  if not tf.gfile.Exists(directory):
    tf.gfile.MakeDirs(directory)

  _, tar_filepath = tempfile.mkstemp(suffix=".tar.gz")

  try:
    tf.logging.info("Downloading %s to %s" % (url, tar_filepath))

    def _progress(count, block_size, total_size):
      sys.stdout.write("\r>> Downloading {} {:.1f}%".format(
          tar_filepath, 100.0 * count * block_size / total_size))
      sys.stdout.flush()

    urllib.request.urlretrieve(url, tar_filepath, _progress)
    print()
    statinfo = os.stat(tar_filepath)
    tf.logging.info(
        "Successfully downloaded %s, size(bytes): %d" % (url, statinfo.st_size))
    with tarfile.open(tar_filepath, "r") as tar:
      tar.extractall(directory)
  finally:
    tf.gfile.Remove(tar_filepath)

In [0]:
def convert_audio_and_split_transcript(input_dir, source_name, target_name,
                                       output_dir, output_file):
  """Convert FLAC to WAV and split the transcript.
  For audio file, convert the format from FLAC to WAV using the sox.Transformer
  library.
  For transcripts, each line contains the sequence id and the corresponding
  transcript (separated by space):
  Input data format: seq-id transcript_of_seq-id
  For example:
   1-2-0 transcript_of_1-2-0.flac
   1-2-1 transcript_of_1-2-1.flac
   ...
  Each sequence id has a corresponding .flac file.
  Parse the transcript file and generate a new csv file which has three columns:
  "wav_filename": the absolute path to a wav file.
  "wav_filesize": the size of the corresponding wav file.
  "transcript": the transcript for this audio segement.
  Args:
    input_dir: the directory which holds the input dataset.
    source_name: the name of the specified dataset. e.g. test-clean
    target_name: the directory name for the newly generated audio files.
                 e.g. test-clean-wav
    output_dir: the directory to place the newly generated csv files.
    output_file: the name of the newly generated csv file. e.g. test-clean.csv
  """

  tf.logging.info("Preprocessing audio and transcript for %s" % source_name)
  source_dir = os.path.join(input_dir, source_name)
  target_dir = os.path.join(input_dir, target_name)

  if not tf.gfile.Exists(target_dir):
    tf.gfile.MakeDirs(target_dir)

  files = []
  tfm = Transformer()
  # Convert all FLAC file into WAV format. At the same time, generate the csv
  # file.
  for root, _, filenames in tf.gfile.Walk(source_dir):
    for filename in fnmatch.filter(filenames, "*.trans.txt"):
      trans_file = os.path.join(root, filename)
      with codecs.open(trans_file, "r", "utf-8") as fin:
        for line in fin:
          seqid, transcript = line.split(" ", 1)
          # We do a encode-decode transformation here because the output type
          # of encode is a bytes object, we need convert it to string.
          transcript = unicodedata.normalize("NFKD", transcript).encode(
              "ascii", "ignore").decode("ascii", "ignore").strip().lower()

          # Convert FLAC to WAV.

          def flac2wav(flac_file):
            wav_path = os.path.join(target_dir, seqid + ".wav")
            song = AudioSegment.from_file(flac_file,format="flac")
            song.export(wav_path, format = "wav")
          flac_file = os.path.join(root, seqid + ".flac")
          #print(flac_file)
          #print(splitext(flac_file)[0])
          wav_file = os.path.join(target_dir, seqid + ".wav")
          print(wav_file)
          if not tf.gfile.Exists(wav_file):
            flac2wav(flac_file)
          wav_filesize = os.path.getsize(wav_file)
          files.append((os.path.abspath(wav_file), wav_filesize, transcript)) 

  # Write to CSV file which contains three columns:
  # "wav_filename", "wav_filesize", "transcript".
  csv_file_path = os.path.join(output_dir, output_file)
  df = pandas.DataFrame(
      data=files, columns=["wav_filename", "wav_filesize", "transcript"])
  df.to_csv(csv_file_path, index=False, sep="\t")
  tf.logging.info("Successfully generated csv file {}".format(csv_file_path))

In [0]:
def download_and_process_datasets(directory, datasets):
  """Download and pre-process the specified list of LibriSpeech dataset.
  Args:
    directory: the directory to put all the downloaded and preprocessed data.
    datasets: list of dataset names that will be downloaded and processed.
  """

  tf.logging.info("Preparing LibriSpeech dataset: {}".format(
      ",".join(datasets)))
  for dataset in datasets:
    tf.logging.info("Preparing dataset %s", dataset)
    dataset_dir = os.path.join(directory, dataset)
    download_and_extract(dataset_dir, LIBRI_SPEECH_URLS[dataset])
    convert_audio_and_split_transcript(
        dataset_dir + "/LibriSpeech", dataset, dataset + "-wav",
        dataset_dir + "/LibriSpeech", 
        dataset + ".csv")


In [36]:
#EXECUTION STARTS FROM HERE
tf.gfile.MakeDirs("/tmp/librispeech_data")
download_and_process_datasets("/tmp/librispeech_data",["train-clean-100"])


INFO:tensorflow:Preparing LibriSpeech dataset: train-clean-100
INFO:tensorflow:Preparing dataset train-clean-100
INFO:tensorflow:Downloading http://www.openslr.org/resources/12/train-clean-100.tar.gz to /tmp/tmpoql2j6hy.tar.gz
>> Downloading /tmp/tmpoql2j6hy.tar.gz 100.0%
INFO:tensorflow:Successfully downloaded http://www.openslr.org/resources/12/train-clean-100.tar.gz, size(bytes): 6387309499
INFO:tensorflow:Preprocessing audio and transcript for train-clean-100


FileNotFoundError: ignored